<a href="https://colab.research.google.com/github/v5v5/trading/blob/main/trade_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URL to download quotes EURUSD

https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true

In [190]:
import pandas as pd
from urllib.request import Request, urlopen
from io import StringIO
import datetime
from dateutil.relativedelta import relativedelta
import time
import random
import math

In [191]:
# dt_from = datetime.date(2000,1,1)
# dt_from = datetime.date(2019,1,1)
dt_from = datetime.date.today() + relativedelta(years=-1)
ut_from = str(int(time.mktime(dt_from.timetuple())))
ut_from

'1588464000'

In [192]:
dt_to = datetime.date.today()
ut_to = str(int(time.mktime(dt_to.timetuple())))
ut_to

'1620000000'

In [193]:
# url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true'
url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=' + ut_from + '&period2=' + ut_to + '&interval=1d&events=history&includeAdjustedClose=true'

In [194]:
headers={'User-Agent': 'Mozilla/5.0'}

In [195]:
req = Request(url, headers=headers)

In [196]:
page = urlopen(req)
# page = urlopen(url)

In [197]:
page = urlopen(url)

In [198]:
content = page.read()

In [199]:
text = content.decode('UTF-8')

In [200]:
print(text[:300])

Date,Open,High,Low,Close,Adj Close,Volume
2020-05-04,1.095746,1.095782,1.090132,1.095963,1.095963,0
2020-05-05,1.090453,1.092132,1.082696,1.090429,1.090429,0
2020-05-06,1.084011,1.084481,1.078423,1.083858,1.083858,0
2020-05-07,1.080054,1.081572,1.076751,1.079855,1.079855,0
2020-05-08,1.083694,1.0873


In [201]:
data = StringIO(text)

In [202]:
df1 = pd.read_csv(data, sep=',', header='infer')

In [203]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume
254,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0
255,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0
256,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0
257,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0
258,2021-04-29,1.213578,1.215214,1.210419,1.213592,1.213592,0
259,2021-04-30,1.212489,1.212856,1.202472,1.212709,1.212709,0
260,2021-05-03,1.202501,1.204094,1.202212,1.202357,1.202357,0


In [204]:
df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')

In [205]:
df1['DayOfWeek'] = df1['Date'].dt.dayofweek

In [206]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
254,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0,4
255,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0,0
256,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0,1
257,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0,2
258,2021-04-29,1.213578,1.215214,1.210419,1.213592,1.213592,0,3
259,2021-04-30,1.212489,1.212856,1.202472,1.212709,1.212709,0,4
260,2021-05-03,1.202501,1.204094,1.202212,1.202357,1.202357,0,0


In [207]:
df2 = df1.drop(['High','Low', 'Adj Close', 'Volume'], axis=1)

In [208]:
df2.tail(7)

,Date,Open,Close,DayOfWeek
254,2021-04-23,1.201793,1.201475,4
255,2021-04-26,1.208956,1.208985,0
256,2021-04-27,1.207921,1.208167,1
257,2021-04-28,1.208854,1.209031,2
258,2021-04-29,1.213578,1.213592,3
259,2021-04-30,1.212489,1.212709,4
260,2021-05-03,1.202501,1.202357,0


In [209]:
df2['Close-1'] = df2['Close'].shift(1)
df2['Open-1'] = df2['Open'].shift(1)

In [210]:
df2.tail(7)

,Date,Open,Close,DayOfWeek,Close-1,Open-1
254,2021-04-23,1.201793,1.201475,4,1.203775,1.203818
255,2021-04-26,1.208956,1.208985,0,1.201475,1.201793
256,2021-04-27,1.207921,1.208167,1,1.208985,1.208956
257,2021-04-28,1.208854,1.209031,2,1.208167,1.207921
258,2021-04-29,1.213578,1.213592,3,1.209031,1.208854
259,2021-04-30,1.212489,1.212709,4,1.213592,1.213578
260,2021-05-03,1.202501,1.202357,0,1.212709,1.212489


In [211]:
df2['%Close'] = (df2['Close'] - df2['Close-1']) / df2['Close-1'] * 100
df2['%Open'] = (df2['Open'] - df2['Open-1']) / df2['Open-1'] * 100

In [212]:
df2.head(7)

,Date,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open
0,2020-05-04,1.095746,1.095963,0,NaN,NaN,NaN,NaN
1,2020-05-05,1.090453,1.090429,1,1.095963,1.095746,-0.504944,-0.483050
2,2020-05-06,1.084011,1.083858,2,1.090429,1.090453,-0.602607,-0.590764
3,2020-05-07,1.080054,1.079855,3,1.083858,1.084011,-0.369329,-0.365033
4,2020-05-08,1.083694,1.083717,4,1.079855,1.080054,0.357641,0.337020
5,2020-05-11,1.084011,1.083952,0,1.083717,1.083694,0.021685,0.029252
6,2020-05-12,1.080917,1.080941,1,1.083952,1.084011,-0.277780,-0.285421


In [220]:
# take decision to buy or sell if price = 1.167072 then buy = -1.167072 or sell = 1.167072
def take_decision(index):
  row = df2.loc[index]
  return random.choice([-row['Open'], +row['Open']])

In [214]:
def is_guessed(index):
  row = df2.loc[index]
  if (index < 1):
    return True
  return (math.copysign(1, row['%Open']) == math.copysign(1, row['decision']))

In [215]:
def make_deal(index):
  k = df2.loc[index, 'k'] 
  decision = df2.loc[index, 'decision'] 
  return decision * (2 ** k)

In [216]:
def discover_trend(index):
  if (index < 1):
    df2.loc[index, 'trend'] = 0
    return
  open_curr = df2.loc[index, '%Open']
  open_prev = df2.loc[index-1, '%Open']
  if (math.copysign(1, open_prev) == math.copysign(1, open_curr)):
    df2.loc[index, 'trend'] = df2.loc[index-1, 'trend'] + 1
  else:
    df2.loc[index, 'trend'] = 0

In [217]:
# this is a main algorithm for trading
def trade():
  for index, row in df2.iterrows():
    df2.loc[index, 'decision'] = take_decision(index)
    if (is_guessed(index)):
      k = 0
    else:
      k += 1
    df2.loc[index, 'k'] = k
    deal = make_deal(index)
    df2.loc[index, 'deal'] = deal
    discover_trend(index)

In [218]:
trade()
df2

,Date,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open,decision,k,deal,trend
0,2020-05-04,1.095746,1.095963,0,NaN,NaN,NaN,NaN,-1.095746,0.0,-1.095746,0.0
1,2020-05-05,1.090453,1.090429,1,1.095963,1.095746,-0.504944,-0.483050,-1.090453,0.0,-1.090453,0.0
2,2020-05-06,1.084011,1.083858,2,1.090429,1.090453,-0.602607,-0.590764,1.084011,1.0,2.168022,1.0
3,2020-05-07,1.080054,1.079855,3,1.083858,1.084011,-0.369329,-0.365033,1.080054,2.0,4.320216,2.0
4,2020-05-08,1.083694,1.083717,4,1.079855,1.080054,0.357641,0.337020,-1.083694,3.0,-8.669552,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
256,2021-04-27,1.207921,1.208167,1,1.208985,1.208956,-0.067660,-0.085611,1.207921,1.0,2.415842,0.0
257,2021-04-28,1.208854,1.209031,2,1.208167,1.207921,0.071513,0.077240,-1.208854,2.0,-4.835416,0.0
258,2021-04-29,1.213578,1.213592,3,1.209031,1.208854,0.377244,0.390783,-1.213578,3.0,-9.708624,1.0
259,2021-04-30,1.212489,1.212709,4,1.213592,1.213578,-0.072759,-0.089735,1.212489,4.0,19.399824,0.0


In [219]:
df2.describe()

,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open,decision,k,deal,trend
count,261.000000,261.000000,261.000000,260.000000,260.000000,260.000000,260.000000,261.000000,261.000000,261.000000,261.000000
mean,1.176550,1.176606,1.992337,1.176507,1.176450,0.036440,0.036557,0.043765,1.164751,0.491680,0.934866
std,0.037332,0.037341,1.419622,0.037378,0.037369,0.400564,0.399199,1.178586,1.503733,18.518845,1.300729
min,1.080054,1.079855,0.000000,1.079855,1.080054,-0.972610,-1.026176,-1.229906,0.000000,-72.562368,0.000000
25%,1.167365,1.167324,1.000000,1.166957,1.167072,-0.252267,-0.246063,-1.181419,0.000000,-2.349514,0.000000
50%,1.184180,1.184273,2.000000,1.184273,1.184178,0.032601,0.029910,1.084011,1.000000,1.127714,0.000000
75%,1.204935,1.205110,3.000000,1.205172,1.205015,0.333199,0.329382,1.185045,2.000000,2.180216,1.000000
max,1.233776,1.234111,4.000000,1.234111,1.233776,1.141047,1.106827,1.233776,7.000000,144.845568,7.000000
